In [6]:
import numpy as np
import math
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import gym
from gym.spaces import Discrete, Tuple

from env_ipd import MetaGames
from rmax import RmaxAgent, Memory

In [7]:
K_epochs = 4  # update policy for K epochs
inner_gamma = 0.9  #inner game discount factor
meta_gamma = 0.99   #meta game discount factor
R_max = 0.98
max_meta_epi = 500
max_meta_steps = 5
max_inner_epi = 10
max_inner_steps = 5
max_visits_per_state = 5
epsilon=0.2
alpha = 0.4
bs = 4          #batch size (must need for mfos for good results)
radius_dp = 1   #no. of decimal point for discretization 

our_rew = torch.zeros(bs).to(device)     #our agent's reward
oppo_rew = torch.zeros(bs).to(device)    #opponent's reward

# creating environment
env = MetaGames(bs, "NL", "IPD")

#action_dim = env.num_actions
nA = env.num_actions
#state_dim = env.num_actions ^ env.d
nS = env.num_actions ** env.d
#no of possible combinations for a inner Q value, #inner_gamma determins the max value of innerq, radius_dp determines number of intervals
poss_combo = math.ceil((10**radius_dp) * (1/(1-gamma)) + 1)


memory = Memory()
rmax = RmaxAgent(env, R_max, meta_gamma, max_meta_epi, max_meta_steps, radius_dp, epsilon, poss_combo)

In [31]:
101**8

10828567056280801

In [21]:
#meta-state = inner game Q table for all agents
meta_s = torch.round(env.innerq, decimals = radius_dp).to(device)
#meta-action = inner game Q table for our agent
meta_a = torch.round(env.innerq[:,:,:,0], decimals = radius_dp).to(device) 

In [23]:
torch.flatten(meta_s[i])

tensor([13.3000,  3.2000, 13.5000,  0.0000, 13.3000,  3.3000, 13.5000,  0.0000,
         0.0000,  0.0000, 13.2000,  3.4000,  0.0000,  3.2000, 13.5000,  1.0000],
       device='cuda:0')

In [29]:
math.ceil(poss_combo)

201

In [ ]:
def find_meta_index(meta, radius_dp, inner_gamma):
    #inner_gamma determins the max value of innerq, radius_dp determines number of intervals
    poss_combo = math.ceil((10 ** radius_dp) * (1/(1-inner_gamma)) +1)
    for i in range(list(meta.size())[0]):
        
        
        
        

In [3]:
for episode in range(rmax.max_episodes): #for each meta-episode
    print("episode =", episode)
    
    for step in range(rmax.max_steps):    #for each meta time step
        print("timestep =", step)
        state = env.reset()   #reset environment 
                   
        for epi in range(10):              #for each inner episodes   
            state = env.reset()   #reset environment 
            for t in range(50):                     #for each inner timestep
                if t == 0:
                    #initialise action by random, of size [agents, batch_size]
                    best_action = torch.randint(2, (env.d, env.b)).to(device)   
                else:
                    #find action that has max Q value for current state for both agents
                    best_action = env.choose_action(state)   

                #run inner game according to that action, for K episodes & T timesteps, output used to be new_state, reward, done, _ 
                newstate, reward, info = env.step(best_action)  

                #update inner r matrix
                for i in range(env.b):
                    env.innerr[i, state[i], best_action[0,i], 0] = reward[i] 
                    env.innerr[i, state[i], best_action[1,i], 1] = info[i]

                #update agents' total reward
                our_rew += reward.reshape(-1)   
                oppo_rew += info.reshape(-1)    

                #update inner q matrix, another for loop since have to wait till inner r matrix gets updated    
                for i in range(env.b):    
                    #env.innerq[i, state[i], best_action[0,i], 0] += alpha * (env.innerr[i, state[i], best_action[0,i], 0] + inner_gamma * torch.max(env.innerq[i, newstate[i], :, 0]) - env.innerq[i, state[i], best_action[0,i], 0])    
                    #env.innerq[i, state[i], best_action[1,i], 1] += alpha * (env.innerr[i, state[i], best_action[1,i], 1] + inner_gamma * torch.max(env.innerq[i, newstate[i], :, 1]) - env.innerq[i, state[i], best_action[1,i], 1])
                    env.innerq[i, state[i], best_action[0,i], 0] = env.innerr[i, state[i], best_action[0,i], 0] + inner_gamma * torch.max(env.innerq[i, newstate[i], :, 0]) 
                    env.innerq[i, state[i], best_action[1,i], 1] = env.innerr[i, state[i], best_action[1,i], 1] + inner_gamma * torch.max(env.innerq[i, newstate[i], :, 1])

                #set current state = new state
                state = newstate  
        
        print("done 1 inner episode,", step)
        #meta-state = inner game Q table for all agents
        meta_s = torch.round(env.innerq, decimals = radius_dp).to(device)
        #meta-action = inner game Q table for our agent
        meta_a = torch.round(env.innerq[:,:,:,0], decimals = radius_dp).to(device)       
        #select meta-action that corresponds to our agent's max Q table
        #best_meta_a = torch.argmax(rmax.Q[:,torch.flatten(meta_s)]).to(device)  
        our_REW = our_rew                           #meta-reward = sum of rewards of our agent in inner game of K episodes & T timesteps
        
        for i in range(env.b):
            rmax.update(memory, find_meta_index(torch.flatten(meta_a[i])) , find_meta_index(torch.flatten(meta_s[i])))
        
#             if done:
#                 if not(reward==1):
#                     self.R[state][best_action]=-10
#                 break


episode = 0
timestep = 0


RuntimeError: size mismatch, got 16, 16x2,4

In [ ]:
#find discretized state index that corresponds to the real state value
                #dstate=[]
                #for j in torch.round(state, decimals = radius_dp):
                #    for i,x in enumerate(ref_arr):
                #        if x==j:
                #            dstate_index.append(i)